In [640]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

import spacy
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import scipy.stats as st

[nltk_data] Downloading package stopwords to /Users/Sanna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [641]:
# containing descriptions of columns
with open('meta_dict.json', 'r') as f: # descriptions of columns
    meta_dict = json.load(f)
print(len(meta_dict))

52


In [726]:
ROOT = Path('/Volumes/INWT/Daten_NLP/') # encrypted folder!
DATA = ROOT / '200707_aachener_zeitung_modified.csv' # text is already minimal preprocessed

In [727]:
df = pd.read_csv(DATA, index_col = 0)
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,titelH3,wordcount,category,city,text_preprocessed,avgTimeOnPage/wordcount,nr_tokens,mean_token_length,nr_tokens_teaser,nr_tokens_titelH1
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,NaN,112.444444,42.857143,33.333333,33.333333,...,NaN,769,vm,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,0.146222,796,5.359296,29,9
48620381,19,6,11,5,1484,NaN,185.500000,42.105263,31.578947,26.315789,...,NaN,441,vm,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",0.420635,452,5.938053,33,8
48622639,2,2,2,2,0,NaN,0.000000,0.000000,100.000000,100.000000,...,NaN,390,vm,Berlin,Eltern auf der Suche nach einem guten Babyphon...,0.000000,396,5.848485,30,7
48623085,32,9,20,9,974,NaN,81.166667,37.500000,28.125000,28.125000,...,NaN,345,vm,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,0.235266,367,5.594005,30,7
48623259,24,2,7,2,3797,NaN,223.352941,70.833333,8.333333,8.333333,...,NaN,182,vm,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,1.227214,183,5.622951,22,8


In [731]:
df = df.fillna('') # replacing Nan with emtpy string
#df.info()

In [733]:
#df.loc[49200513]

In [724]:
df.loc[49200513, 'text_preprocessed'] # one example

'Die neuen Fälle von Ansteckungen mit dem Coronavirus Sars-CoV-2 haben zum Teil Konsequenzen für öffentliche Einrichtungen. In Nordrhein-Westfalen sind als Vorsichtsmaßnahme gegen die Krankheit Covid-19 einzelne Kindergärten und Schulen geschlossen, auf Teneriffa sitzen Urlauber in einem Hotel fest. Was gilt für Arbeitgeber in solchen Situationen - und was sollten Beschäftigte noch dazu wissen? Fall 1: Die Kita oder Schule bleibt geschlossen Wenn Kindergärten und Schulen aus Vorsicht zum Infektionsschutz geschlossen sind, können Arbeitnehmer im Notfall zu Hause bleiben, um ihre Kinder zu betreuen, erklärt Nathalie Oberthür, Fachanwältin für Arbeitsrecht aus Köln. Ob sie dann auch weiter ihr Gehalt bekommen, hängt aber davon ab, ob wirklich keine andere Betreuung möglich war. Paragraf 616 im Bürgerlichen Gesetzbuch (BGB) besagt, dass, wer ohne eigenes Verschulden und aus einem persönlichen Grund verhindert ist und nicht zur Arbeit kommen kann, trotzdem weiter Gehalt bekommt. Die erste M

In [734]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 48620281 to 51565349
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   pageviews                891 non-null    int64  
 1   entrances                891 non-null    int64  
 2   exits                    891 non-null    int64  
 3   bounces                  891 non-null    int64  
 4   timeOnPage               891 non-null    int64  
 5   conversions              891 non-null    object 
 6   avgTimeOnPage            891 non-null    float64
 7   stickiness               891 non-null    float64
 8   entranceRate             891 non-null    float64
 9   bounceRate               891 non-null    float64
 10  kategorie                891 non-null    object 
 11  rubrik                   891 non-null    object 
 12  subRubrik                891 non-null    object 
 13  titel                    891 non-null    object 
 14  pagePath      

In [649]:
#df.sort_values(by='pageviews', ascending=False)
df.sort_values(by='timeOnPage', ascending=False)
#df.sort_values(by='avgTimeOnPage', ascending=False)
#df.sort_values(by='stickiness', ascending=False)

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,text,titelH1,titelH3,wordcount,category,city,text_preprocessed,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
articleId,,,,,,,,,,,,,,,,,,,,,
49200513,3047,1119,1693,924,217337,,160.514771,44.437151,36.724647,30.324910,...,Köln (dpa/tmn) - Die neuen Fälle von Ansteckun...,Einschränkungen wegen Covid-19: Was Arbeitnehm...,,376,vm,Köln,Die neuen Fälle von Ansteckungen mit dem Coron...,379,5.598945,0.426901
49473155,1887,712,1006,587,169090,,191.929625,46.687864,37.731849,31.107578,...,Düsseldorf (dpa/tmn) - Um die Verbreitung des ...,Veranstaltung wegen Corona abgesagt: Rechte de...,,708,vm,Düsseldorf,Um die Verbreitung des neuen Coronavirus Sars-...,718,5.988858,0.271087
49333693,2487,1877,2032,1699,136927,,300.938462,18.295135,75.472457,68.315239,...,Frankfurt/Main/Düsseldorf (dpa/tmn) - Nachdem ...,Covid-19: Welche Regeln gelten im Betrieb?,,768,vm,Frankfurt/Main/Düsseldorf,Nachdem mehr und mehr Fälle von Covid-19-Infek...,770,6.137662,0.391847
49321583,1709,1226,1333,1085,92555,,246.156915,22.001170,71.737858,63.487420,...,Berlin (dpa/tmn) - Der neuartige Coronavirus S...,Ansteckung und Vorbeugung: Der Alltag mit Covi...,,882,vm,Berlin,Der neuartige Coronavirus Sars-Cov-2 verbreite...,889,5.823397,0.279089
49232551,1548,580,811,468,77052,,104.548168,47.609819,37.467700,30.232558,...,Berlin (dpa/tmn) - Eine Impfung gegen das neue...,Coronavirus: Experten raten zu Impfungen - auc...,,253,vm,Berlin,Eine Impfung gegen das neue Coronavirus Sars-C...,252,6.178571,0.413234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48870125,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,Frankfurt/Main (dpa/tmn) - Wonach sehnen Sie s...,Sinnbild für Exotik: Der Affe im Dschungel als...,,415,vm,Frankfurt/Main,Wonach sehnen Sie sich? Den Wald im regnerisch...,427,5.341920,0.000000
49133249,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,Berlin (dpa/tmn) - Wenn die Schönheitsreparatu...,Schönheitsreparaturen auch in möblierten Wohnu...,,148,vm,Berlin,Wenn die Schönheitsreparaturen in einer Wohnun...,143,6.265734,0.000000
49571165,21,21,21,21,0,,0.000000,0.000000,100.000000,100.000000,...,Cottbus/Berlin (dpa/tmn) - Ein Job am Schreibt...,Bei Wind und Wetter im Einsatz: Die Ausbildung...,,638,vm,Cottbus/Berlin,"Ein Job am Schreibtisch, das wäre nichts für A...",650,5.755385,0.000000


## Modeling pageviews / avgTimeOnPage with (lemmatized) Bag-of-Words (Logistic Regression)

In [650]:
#creating train, dev, test
RANDOM_SEED = 123
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, shuffle=True)
print(df_train.shape, df_dev.shape, df_test.shape)

(712, 33) (89, 33) (90, 33)


In [651]:
# lemmatizes (punctuation stays in, but the default tokenizer from Count_Vectorizer seems to delete it)
class Preprocessor():
    def __init__(self):
        self.nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])
        #self.nlp = spacy.load("de_core_news_md", disable=['parser', 'ner'])

    def __call__(self, doc):
        rt = []
        doc = self.nlp(doc)
        for token in doc:
            rt.append(token.lemma_.lower()) # stopwords stay in, but CountVectorizer uses nltk-list later

        return " ".join(rt)

In [652]:
stopwords = nltk.corpus.stopwords.words('german')
print(stopwords[:10])
print(len(stopwords))
stopwords = [ preprocessor(s) for s in stopwords ]
print(stopwords[:10])
print(len(stopwords))

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an']
232
['aber', 'all', 'alle', 'alle', 'all', 'alle', 'als', 'also', 'am', 'an']
232


In [653]:
preprocessor = Preprocessor()
vectorizer = CountVectorizer(analyzer='word',
                             preprocessor=preprocessor,
                             ngram_range=(1, 5),
                             min_df=3, # Achtung: vielleicht je nach Textbasis anpassen?
                             max_features=10000,
                             stop_words=stopwords
                            )

In [699]:
# define text base features: text? text_preprocessed? titelH1?

#feature = 'text_preprocessed'
#feature = 'text'
feature = 'titelH1'
#feature = 'teaser' # note: not all have 'teaser' but I replaced Nan wih empty string ""

X_train = vectorizer.fit_transform(df_train[feature])

In [700]:
vectorizer.get_feature_names()[100:120] # just some n-grams

['herzpatienten',
 'hilfe',
 'hoch',
 'homeoffice',
 'immer',
 'immobilie',
 'isolation',
 'jahr',
 'job',
 'kartoffel',
 'kauf',
 'kaufen',
 'kind',
 'kita',
 'kleine',
 'kleiner',
 'knolle',
 'kochen',
 'kommen',
 'kosten']

In [701]:
X_dev = vectorizer.transform(df_dev[feature])
X_test = vectorizer.transform(df_test[feature])

In [702]:
X_train, X_dev, X_test = X_train.toarray(), X_dev.toarray(), X_test.toarray()

In [703]:
X_train.shape, X_dev.shape, X_test.shape

((712, 255), (89, 255), (90, 255))

In [704]:
# define the target labels
#target = 'timeOnPage'
target =  'pageviews'
#target = 'avgTimeOnPage'
#target = 'stickiness'

y_train = np.array(df_train[target])
y_dev = np.array(df_dev[target])
y_test = np.array(df_test[target])

In [705]:
y_train.shape, y_dev.shape, y_test.shape

((712,), (89,), (90,))

In [706]:
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor # LogisticRegression

In [707]:
# choose model ???
model = Ridge()
#model = LinearRegression()
#model = SGDRegressor()
#model = LogisticRegression(max_iter=10000)

In [708]:
model.fit(X_train, y_train)

Ridge()

In [709]:
# predict for dev set
pred_dev = model.predict(X_dev)

In [710]:
pred_dev

array([ 1.10011763e+03,  3.89209939e+01,  2.03498260e+01,  3.09304282e+01,
       -6.53767184e-01,  1.57286332e+01, -3.48198942e+00,  5.68710048e+01,
        8.13209927e+00,  1.42045028e+01,  1.81033800e+00,  1.62218525e+02,
        1.42045028e+01,  1.81015584e+01,  6.95019161e+01,  1.42045028e+01,
       -4.82301940e+00, -3.26347085e+01,  1.25541607e+01,  1.81384638e+02,
        1.81384638e+02,  1.97287164e+01, -2.02028577e+01,  1.42045028e+01,
        9.03860432e+01,  1.25210239e+01,  6.90729752e+01,  1.25212142e+01,
        1.42045028e+01,  3.87830098e+01, -5.76724517e+01,  1.23984097e+01,
        1.42045028e+01, -3.67251919e+01,  1.82399598e+01,  4.03643511e+01,
        2.52690094e+00,  4.33435444e+01,  1.42045028e+01, -2.07594332e+01,
        2.44289639e+01,  6.05266562e+01,  4.90875231e+00, -9.25439930e+00,
       -2.15494836e+01,  7.16772717e+00,  4.81424580e+00,  1.52849882e+02,
        1.61764672e+01,  1.02604378e+01,  1.62255440e+01,  4.09457993e+00,
        1.57286332e+01, -

In [711]:
# postprocessing: replace negative values with 0 (better way? can I give that hint to the model?)
pred_dev[pred_dev < 0] = 0

In [712]:
pred_dev

array([1100.11762759,   38.92099387,   20.34982595,   30.93042817,
          0.        ,   15.72863323,    0.        ,   56.87100484,
          8.13209927,   14.20450283,    1.810338  ,  162.21852478,
         14.20450283,   18.10155835,   69.50191611,   14.20450283,
          0.        ,    0.        ,   12.55416072,  181.38463796,
        181.38463796,   19.72871635,    0.        ,   14.20450283,
         90.38604324,   12.52102393,   69.07297517,   12.5212142 ,
         14.20450283,   38.78300979,    0.        ,   12.39840969,
         14.20450283,    0.        ,   18.23995977,   40.36435106,
          2.52690094,   43.34354435,   14.20450283,    0.        ,
         24.42896394,   60.52665624,    4.90875231,    0.        ,
          0.        ,    7.16772717,    4.8142458 ,  152.84988219,
         16.17646716,   10.26043783,   16.22554395,    4.09457993,
         15.72863323,    0.        ,   14.20450283,    0.        ,
         14.67462912,   47.18303731,   16.17646716,    5.72291

In [713]:
np.array(y_dev)

array([3047,   33,   20,   64,    1,   12,    2,   22,    3,    6,    1,
         19,    3,  262,    9,    4,   24,  100,   27,  124,    3,    8,
         14,    6,   39,   14,   16,   18,    9,    8,    8,    9,   16,
         32,   19,    8,   20,    3,   18,   11,   18,    9,    9,   24,
         14,    6,    5,  115,    4,   14,    9,    7,   16,   17,    6,
          8,   19,   39,   17,   29,   24,   13,    1,    8,   10,    3,
          7,   14,    9,    7,    7,    2,    7,    4,   12,    5,   12,
          7,  173,   15,    3,   18,   29,   21,    1,   10,    2,    9,
         18])

In [714]:
st.pearsonr(pred_dev, y_dev)

(0.8613052175134238, 2.5749805968516432e-27)

In [715]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

In [716]:
mean_squared_error(pred_dev, y_dev) #, squared=False) # hard to interpret

48101.96843497607

In [717]:
mean_absolute_error(pred_dev, y_dev) # hard to interpret

54.62542091026355

In [718]:
explained_variance_score(pred_dev, y_dev) # what does this number mean? ... (1.0 is best)

-1.8341015949405

In [676]:
df_dev.iloc[4] # Example where timeOnPage = 0 (boring article! :D)

pageviews                                                                  1
entrances                                                                  0
exits                                                                      1
bounces                                                                    0
timeOnPage                                                                 0
conversions                                                                 
avgTimeOnPage                                                              0
stickiness                                                                 0
entranceRate                                                               0
bounceRate                                                                 0
kategorie                                                           ratgeber
rubrik                                                                reisen
subRubrik                                                                   

In [192]:
#vectorizer.stop_words_ # shows not used features (in too many/little documents)
#vectorizer.vocabulary_ # values are the indices! (not frequency or anything)

## Beobachtung

Bei timeOnPage ist das Modell sehr gut, bei pageviews allerings nicht so sehr. Woran liegt das? Lernt das Modell vielleicht eher anhand der reinen Textlänge und nicht spezifische Wörter? Hier wäre vielleicht Shap gut...

Bei pageviews vielleicht eher den Titel oder Teaser als den Text selbst nehmen? (--> etwas besser, dann aber min_df ändern) --> Titel ist besser als Teaser (vielleicht auch sinnvoller?)

Welches Label ist das sinnvollste?

Welche Metrik? Pearson Korrelation? Absolute Error?

Vielleicht mal einfach die Keywords, Subjekts aufsplitten und als Features nehmen?

Problem: Target-Varieblen sind sehr unausgeglichen, wenige Artikel mit sehr hoher pageview-Zahl (outliers)